In [1]:
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from torch import nn


The primary source of inspiration on this solution may be found here: https://towardsdatascience.com/text-classification-with-bert-in-pytorch-887965e5820f

The major differences being this article is outdated and BERT has moved locations, using hugging face to download the tokenizer and model

In [23]:
#!pip install tqdm boto3 requests regex sentencepiece sacremoses importlib_metadata
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')    # Download vocabulary from S3 and cache.
model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-uncased')    # Download model and configuration from S3 and cache.


Using cache found in C:\Users\shurygin/.cache\torch\hub\huggingface_pytorch-transformers_main
Using cache found in C:\Users\shurygin/.cache\torch\hub\huggingface_pytorch-transformers_main
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertF

In [19]:
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [20]:
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [21]:
print(max([len(tx) for tx in data["text"]])) #the max character length of the text columns

157


In [7]:
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [26]:
bert_input_example = tokenizer(data["text"][0],padding='max_length', max_length = 160,  
                       truncation=True, return_tensors="pt") #twitter max size of any tweet is around 160 characters, so lets use 160 as our max length

In [31]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.labels = df["target"].values
        self.text = [tokenizer(text, padding='max_length', max_length = 160,  
                       truncation=True, return_tensors="pt") for text in df["text"]]
    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.text[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [32]:
seed = 123
df_train, df_val = train_test_split(data, test_size=0.2, random_state = seed)

In [33]:
df_train

,id,keyword,location,text,target
4321,6135,hijack,Houston TX,Tension In Bayelsa As Patience Jonathan Plans ...,1
6107,8720,sinking,"Sacramento, CA",So happy to be exercised of the demon of @ATT....,0
4973,7091,meltdown,The Internet,Byproduct of metal price meltdown is a higher ...,1
6774,9706,tornado,"Kentucky, USA",Tornado has to make the playoffs. They have 3 ...,0
5768,8235,riot,"Los Angeles, CA",@Trollkrattos Juan Carlos Salvador The Secret ...,0
...,...,...,...,...,...
4395,6247,hijacking,World,The Murderous Story Of AmericaÛªs First Hijac...,1
126,182,aftershock,304,'The only thing that stands between you and yo...,0
5162,7361,obliterate,Cymru araul,@McCaineNL Think how spectacular it will look ...,1
1773,2546,collision,"Denver, Colorado",Motorcyclist bicyclist injured in Denver colli...,1


In [35]:

class BertDisasterClassifier(nn.Module):

    def __init__(self, dropout = 0.2):
        super(BertDisasterClassifier, self).__init__()
        self.bert = model
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU() #we tried Relu and Softmax here
    
    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask, return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        return final_layer
        #return linear_output

In [38]:
#the basic training scheme I took from the data science medium article because they mostly look like this anyways.
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)
    

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=1, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):
                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)
                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')

In [39]:
EPOCHS = 5
model_augmented = BertDisasterClassifier()
LR = 1e-6 #max lambda for ADAM to cap out, could be higher for faster converging 
               
train(model_augmented, df_train, df_val, LR, EPOCHS)

  5%|▌         | 315/6090 [05:16<1:36:48,  1.01s/it]


KeyboardInterrupt: 

In [ ]:
model_augmented.eval()
test_encoded = [tokenizer(text, padding='max_length', max_length = 160, truncation=True, return_tensors="pt") for text in test["text"]]
test_ids = test["id"]


In [ ]:
#could definitely make this better however the file is only 3000 predictions, took around 25 seconds on GPU's
predicted_list = []
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
for datapoint in test_encoded:
    mask = datapoint['attention_mask'].to(device)
    input_id = datapoint['input_ids'].squeeze(1).to(device)
    output = model_augmented(input_id, mask)
    answer = np.argmax(output.to("cpu").detach().numpy() )
    predicted_list.append(answer)
    

In [ ]:
output = pd.DataFrame()
output["id"] = test_ids
output["target"] = predicted_list
output.to_csv("submission_BERT_linear_relu_head.csv", index=False)

After running this notebook on Kaggle notebooks and using GPUs to accelerate training we recieved a final test accuracy of __82.8%__ with validation accuracies hovering around that point for all 5 epochs trained.

The Training Loss and Accuracy for each epoch are as follows:
Epochs: 1 | Train Loss:  0.529                 | Train Accuracy:  0.739                 | Val Loss:  0.436                 | Val Accuracy:  0.813

Epochs: 2 | Train Loss:  0.361                 | Train Accuracy:  0.854                 | Val Loss:  0.419                 | Val Accuracy:  0.823

Epochs: 3 | Train Loss:  0.290                 | Train Accuracy:  0.895                 | Val Loss:  0.436                 | Val Accuracy:  0.820

Epochs: 4 | Train Loss:  0.230                 | Train Accuracy:  0.919                 | Val Loss:  0.465                 | Val Accuracy:  0.821

Epochs: 5 | Train Loss:  0.175                 | Train Accuracy:  0.940                 | Val Loss:  0.491                 | Val Accuracy:  0.816

With a training time of around 35 minutes total

At the time of finishing this places me 145th out of 929 teams, or in the top __~15%__